In [1]:
import os
import shutil
from collections import defaultdict
import cv2

# import SimpleITK as sitk
import collections

from pathlib import Path
import numpy as np
import gc

import pandas as pd
import polars as pl
import pydicom
import ipywidgets as widgets
import glob
from scipy.ndimage import zoom
from scipy import ndimage

import csv
import copy

from functools import lru_cache

import torch
import torch.cuda
from torch.utils.data import Dataset

import kaggle_evaluation.rsna_inference_server

ModuleNotFoundError: No module named 'kaggle_evaluation'

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in `rsna_gateway` will run in a different container with direct access to the hidden test set and hand off the data series by series.

Your code will always have access to the published copies of the files.

In [6]:
ID_COL = 'SeriesInstanceUID'

LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

# All tags (other than PixelData and SeriesInstanceUID) that may be in a test set dcm file
DICOM_TAG_ALLOWLIST = [
    'BitsAllocated',
    'BitsStored',
    'Columns',
    'FrameOfReferenceUID',
    'HighBit',
    'ImageOrientationPatient',
    'ImagePositionPatient',
    'InstanceNumber',
    'Modality',
    'PatientID',
    'PhotometricInterpretation',
    'PixelRepresentation',
    'PixelSpacing',
    'PlanarConfiguration',
    'RescaleIntercept',
    'RescaleSlope',
    'RescaleType',
    'Rows',
    'SOPClassUID',
    'SOPInstanceUID',
    'SamplesPerPixel',
    'SliceThickness',
    'SpacingBetweenSlices',
    'StudyInstanceUID',
    'TransferSyntaxUID',
]

# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each prediction (except the very first) must be returned within 30 minutes of the series being provided.
def predict(series_path: str) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # --------- Replace this section with your own prediction code ---------
    series_id = os.path.basename(series_path)
    
    all_filepaths = []
    for root, _, files in os.walk(series_path):
        for file in files:
            if file.endswith('.dcm'):
                all_filepaths.append(os.path.join(root, file))
    all_filepaths.sort()
    
    # Collect tags from the dicoms
    tags = defaultdict(list)
    tags['SeriesInstanceUID'] = series_id
    global dcms
    for filepath in all_filepaths:
        ds = pydicom.dcmread(filepath, force=True)
        tags['filepath'].append(filepath)
        for tag in DICOM_TAG_ALLOWLIST:
            tags[tag].append(getattr(ds, tag, None))
        # The image is in ds.PixelData

    # ... do some machine learning magic ...
    predictions = pl.DataFrame(
        data=[[series_id] + [0.5] * len(LABEL_COLS)],
        schema=[ID_COL, *LABEL_COLS],
        orient='row',
    )
    # ----------------------------------------------------------------------

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == [ID_COL, *LABEL_COLS]
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == [ID_COL, *LABEL_COLS]).all()
    else:
        raise TypeError('The predict function must return a DataFrame')

    # ----------------------------- IMPORTANT ------------------------------
    # You MUST have the following code in your `predict` function
    # to prevent "out of disk space" errors. This is a temporary workaround
    # as we implement improvements to our evaluation system.
    shutil.rmtree('/kaggle/shared', ignore_errors=True)
    # ----------------------------------------------------------------------
    
    return predictions.drop(ID_COL)

When your notebook is run on the hidden test set, `inference_server.serve` must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first `predict` call.

In [7]:
inference_server = kaggle_evaluation.rsna_inference_server.RSNAInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway()
    display(pl.read_parquet('/kaggle/working/submission.parquet'))

SeriesInstanceUID,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1.2.826.0.1.3680043.8.498.1005…",0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
"""1.2.826.0.1.3680043.8.498.1002…",0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
"""1.2.826.0.1.3680043.8.498.1007…",0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


# 데이터 경로 정리

In [9]:
DATA_PATH = '/kaggle/input/rsna-intracranial-aneurysm-detection'
SERIES_PATH = os.path.join(DATA_PATH,'series')
SEG_PATH = os.path.join(DATA_PATH,'segmentations')
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
train_localizers_df = pd.read_csv(os.path.join(DATA_PATH, 'train_localizers.csv'))

In [10]:
# 검증용 데이터 분리 
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3478 entries, 2804 to 2853
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   SeriesInstanceUID                           3478 non-null   object
 1   PatientAge                                  3478 non-null   int64 
 2   PatientSex                                  3478 non-null   object
 3   Modality                                    3478 non-null   object
 4   Left Infraclinoid Internal Carotid Artery   3478 non-null   int64 
 5   Right Infraclinoid Internal Carotid Artery  3478 non-null   int64 
 6   Left Supraclinoid Internal Carotid Artery   3478 non-null   int64 
 7   Right Supraclinoid Internal Carotid Artery  3478 non-null   int64 
 8   Left Middle Cerebral Artery                 3478 non-null   int64 
 9   Right Middle Cerebral Artery                3478 non-null   int64 
 10  Anterior Communicating Art

# train 데이터 그래프
- 연령 : 50 ~ 70대 다수
- 동맥류 없는 경우가 훨씬 많음


In [ ]:
train_df.hist(figsize=(20,20))

In [ ]:
train_df.head()

In [ ]:
train_localizers_df.info()

In [ ]:
train_localizers_df.head()
# 여기서 왜 coordinates가 x와 y정보만 있을까? 

# 3. DICOM series 불러오기 및 3D volumn 만들기 

## 이 과정의 의미와 필요성

**DICOM 시리즈 불러오기 및 3D 볼륨 만들기**는 의료 영상(특히 CT, MRI 등)에서 여러 장의 2D 슬라이스 이미지를 순서대로 불러와서, 이를 하나의 3차원(3D) 데이터로 합치는 과정이야. 

### 왜 필요한가?
- **의료 영상 데이터는 대부분 2D 슬라이스(얇게 자른 단면 이미지)로 저장**돼 있어. 한 환자의 CT 검사 결과는 수십~수백 장의 DICOM 파일(각각 한 슬라이스)로 제공됨.[1][6]
- 이 슬라이스들을 올바른 순서로 쌓으면, **3D 볼륨(입체 영상)**을 만들 수 있어. 이렇게 하면 실제 인체 구조를 입체적으로 분석하거나, 3D로 시각화, 딥러닝 모델 입력 등 다양한 분석이 가능해.[2][3][4]
- 예를 들어, 뇌 동맥류 위치를 3D로 파악하거나, 3D 마스크와 비교할 때 꼭 필요해.

## 과정 요약
1. **폴더에서 모든 DICOM 파일(슬라이스) 불러오기**
2. **슬라이스 순서대로 정렬** (보통 파일명 또는 DICOM 메타데이터의 위치 정보 활용)
3. **각 슬라이스의 픽셀 데이터를 3D 배열로 쌓기**

***

## 코드 예시와 설명

```python
import os
import pydicom
import numpy as np

# 1. 시리즈 폴더 경로 지정 (실제 SeriesInstanceUID로 변경)
series_dir = 'series/1.2.826.0.1.3680043.12345'

# 2. 폴더 내 모든 DICOM 파일 경로 리스트업
files = [os.path.join(series_dir, f) for f in os.listdir(series_dir) if f.endswith('.dcm')]

# 3. DICOM 파일을 모두 읽어서 리스트로 저장
slices = [pydicom.dcmread(f) for f in files]

# 4. (선택) 슬라이스를 위치 정보로 정렬 (SliceLocation, InstanceNumber 등)
slices = sorted(slices, key=lambda s: float(s.InstanceNumber))

# 5. 각 슬라이스의 픽셀 데이터를 3D 배열로 쌓기
volume = np.stack([s.pixel_array for s in slices])
print('3D 볼륨 shape:', volume.shape)  # (슬라이스 수, 높이, 너비)
```

### 코드 설명
- **1~2단계:** 한 시리즈 폴더(한 번의 CT/MRI 촬영 결과)에서 모든 DICOM 파일을 찾음.
- **3단계:** 각 파일을 pydicom으로 읽어서 DICOM 객체 리스트로 만듦.
- **4단계:** 슬라이스가 올바른 순서(머리→발, 앞→뒤 등)로 쌓이도록 정렬. InstanceNumber, SliceLocation 등 메타데이터를 활용함.[6]
- **5단계:** 각 슬라이스의 픽셀 데이터를 numpy 배열로 추출해서, (슬라이스 수, 높이, 너비) 형태의 3D 볼륨을 만듦.

***

## 요약
- 이 과정은 **2D 슬라이스를 3D 입체 영상으로 재구성**하는 핵심 단계야.
- 3D 볼륨이 있어야 실제 인체 구조 분석, 3D 시각화, 딥러닝 모델 입력 등 다양한 의료 영상 분석이 가능해진다.[3][4][1][2][6]

In [ ]:
path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'


dataset = pydicom.dcmread(path)
pixel_array = dataset.pixel_array

In [ ]:
import pydicom
import numpy as np
import os
import matplotlib.pyplot as plt


series_dir = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647'

# 슬라이스 파일 정렬 및 읽기
files = sorted([os.path.join(series_dir, f) for f in os.listdir(series_dir) if f.endswith('.dcm')])
slices = [pydicom.dcmread(f) for f in files]

# 3D 볼륨 생성
volume = np.stack([s.pixel_array for s in slices])
print('3D 볼륨 shape:', volume.shape)

In [ ]:
# 가운데 슬라이스 시각화
mid = len(volume) // 2
plt.imshow(volume[mid], cmap='gray')
plt.title('Middle Slice')
plt.axis('off')
plt.show()

In [ ]:
# DICOM 파일 시리즈 읽기
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(series_dir)
reader.SetFileNames(dicom_names)

# DICOM 시리즈를 읽어 이미지로 변환
image = reader.Execute()

# 이미지 정보 확인
print(f"Image size: {image.GetSize()}")   # 이미지의 크기
print(f"Pixel spacing: {image.GetSpacing()}")  # 픽셀 간격


# numpy 배열로 변환
image_array = sitk.GetArrayFromImage(image)
print(image_array.shape)  # numpy 배열의 크기

In [ ]:
plt.imshow(image_array[0], cmap='gray')
plt.show()

In [ ]:
path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'

dicom_image = sitk.ReadImage(path)

# 1. 원점 가져오기
origin = dicom_image.GetOrigin()
print("원점", origin) 
# 원점 (-83.338355229404, -79.849251419771, 38.935824681372)

# 2. 크기 가져오기
size = dicom_image.GetSize()
print("크기 : ", size) # 크기 :  (512, 512, 1)

# 3. 깊이 가져오기
depth = dicom_image.GetDepth()
print('깊이:',depth) # 깊이: 1

# 4. 특정 위치의 픽셀 값 가져오기
pixel_location = (0,0,0)
pixel_value = dicom_image.GetPixel(*pixel_location)

print('픽셀 값 (0,0,0):', pixel_value) # 픽셀 값 (0,0,0): 0

# 5. 원점 설정하기 
new_origin = [0.0, 0.0, 0.0]
dicom_image.SetOrigin(new_origin)

print('새 원점 설정 완료', new_origin)

### 동맥류 여부 변수, x,y, series_uid를 담을 튜플 정의 

In [ ]:
CandidateInfoTuple = collections.namedtuple('CandidateInfoTuple',
                                            'is_aneurysm_present, series_uid, sop_uid, center_xyz')

## serise/ 내의 series_uid 폴더 별 dicom 이미지를 3D array로 변환

In [16]:
class DICOMPreprocessor:   

    def __init__(self, target_shape = (16, 128, 128)):
        self.target_depth , self.target_height, self.target_width = target_shape
    
    """
       Load DICOM series
       """
    
    @lru_cache(1) # @lru_cache(1) 수의 결과를 캐싱해준다. 호출 결과가 이미 캐싱되어있으면 함수를 호출하지않고 캐싱된 결과를 반환한다.
    def load_dicom_series(self, series_path):
        dicom_files = []

        series_path = Path(series_path)
        series_name = series_path.name 
        
        # os.walk : 모든 하위 파일까지 다 탐색 가능 , (경로, 경로 내 디렉토리 리스트, 경로 내 파일 리스트)
    
        for root, _ , files in os.walk(series_path): 
            for file in files:
                if file.endswith('.dcm'):
                    dicom_files.append(os.path.join(root,file))
        if not dicom_files:
            raise ValueError(f"no Dicom files found in {series_path}")
        print(f"Found {len(dicom_files)} DICOM files in series {series_path}")
    
        # Load DICOM datasets
        datasets = []
    
        for filepath in dicom_files:
            try:
                ds = pydicom.dcmread(filepath)
                datasets.append(ds)
            except Exception as e:
                print(f"Failed to load {filepath}: {e}")
                continue
        if not datasets : 
            raise ValueError(f"No valid DICOM files in {series_path}")
        return datasets , series_name
    
    
    """
    Extract position information for each slice 
    """
    def extract_slice_info(self, datasets):
        slice_info = []
        for i, ds in enumerate(datasets):
            info = {
                'dataset' : ds,
                'index' : i,
                'instance_number' : getattr(ds, 'InstanceNumber',i)
            }
    
    # Get z-coordinate from ImagePositionPatient and ImageOrientationPatient
            try:
                ipp = np.array(getattr(ds, 'ImagePositionPatient',None))
                iop = np.array(getattr(ds, 'ImageOrientationPatient', None))
                n_vec = np.cross(iop[:3],iop[3:])
                if iop is None:
                    info['z_position'] = float(i)
                else:
                    info['z_position'] = -float((ipp*n_vec).sum())
            except Exception as e:
                info['z_position'] = float(i)
                print(f"failed to extract position info, {i}: {e}")
            slice_info.append(info)
        return slice_info
    
    """
    Sort slices by z-coordinate
    """
    def sort_slices_by_position(self, slice_info):
        sorted_slices = sorted(slice_info, key = lambda x : x['z_position'])
        return sorted_slices
    
    
    '''
    Get windowing parameters based on modality
    - CT 이미지에 대해서만 윈도잉을 하는 이유 
    windowing : CT 영상에서 나타나는 선형 밀도(HU)를 영상의 특정 범위(윈도우)로 제한해 명암대비를 조절하는 과정
    전체 HU 범위를 영상으로 그대로 표현하면, 눈으롱 인지하기 어려울 정도의 넓은 명암 범위가 생성됨
    특정 관심 조직을 부각하기 위해서는 윈도우 폭과 중심을 조절해 해당 조직의 HU 범위를 적절히 드러냄
    
    '''
    def get_windowing_params(self, ds:pydicom.Dataset, img:np.ndarray=None) :
        modality = getattr(ds,'Modality','CT') 
    
        if modality == 'CT':
            # For CT , apply CTA settings
            # center : widnow level (윈도우 중심)
            # 50인 경우 HU 50을 기준으로 대칭적인 구간 50 +- width/2를 윈도우 범위로 설정
            # 폭이 넓으면 대비가 낮아지고, 폭이 조으면 대비가 높아진다. 
            # width : window width (윈도우 폭)
            # 여기서는 -125 ~ 225 사이의 구간만 명암을 세밀하게 표현한다. 
            center, width = (40,350)
            
            return center, width
            # return "CT","CT"
        elif modality =='MR':
            return None, None, # MR 영상은 윈도잉 없이 통계적 정규화만 할 것 ! 
    
        else:
            return None,None 
    
    '''
    Extract 2D pixel array from DICOM 
    input : ds
    output : 한 파일에 여러 슬라이스가 있을 경우, 즉 3D일 경우 가운데 슬라이스 선택, 컬러일경우 grayscale로 변경, 이 후 HU단위로 변경 적용 후 img 반환 
    '''
    
    def extract_pixel_array(self, ds):
        img = ds.pixel_array.astype(np.float32)
    
        # For 3D volume case (multiple frame ) - select middle frame
        if img.ndim == 3:
            frame_idx = img.shape[0] // 2
            img = img[frame_idx]
            print(f"Selected frame {frame_idx} from 3D DICOM")
        
        # Convert color image to grayscale
        # img.shape[-1] 이미지의 마지막 차원의 크기 가져오기 
        if img.ndim ==3 and img.shape[-1]==3:
            img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2GRAY).astype(np.float32)
            # print("Converted color image to grayscale")
    
        '''
        Apply RescaleSlope and RescaleIntercept 
        DICOM 파일은 저장공간과 호환성 문제로 인해 픽셀 값을 단순한 정수로 저장한다. 
        그러나 실제 의료 영상에서 의미있는 값(HU)을 얻으려면 변환을 해줘야한다.
        - RescaleSlope ,RescaleIntercept 는 DICOM 헤더에 저장된 선형 변환 계수이다.
        - 실제 물리적 단위로 변환하는 공식 : 
            Image value = (RescaleSlope) * (Stored value) + RescaleIntercept
        - 이 변환을 적용하지 않으면, 영상의 밝기와 대조가 실제의 물리적 특성을 반영하지 못한다.
        예를 들어서 CT에서 물은 HU = 0, 공기는 HU = -1000, 뼈는 HU = 400~ 1000등으로 구분되는데, 변환하지 않으면 이런 구분이 불가능하다.
        
        ''' 
        slope = getattr(ds, 'RescaleSlope',1)
        intercept = getattr(ds,'RescaleIntercept',0)
        if slope != 1 or intercept != 0:
            img = img * float(slope) + float(intercept)
            # print(f"Applied rescaling: slope={slope}, intercept={intercept}")
    
        return img
    
    '''
    Apply windowing or statistical normalization
    input : img, window_center, window_width
    output : CT - windowing, MRI - normalization 적용 후 uint8로 변환 후 반환 
    
    '''
    def apply_windowing_or_normalize(self,img,center,width):
        if center is not None and width is not None :
    
            # 전통적 방식의 CT windowing
            # Windowing processing (for CT/CTA)
            img_min = center - width / 2
            img_max = center + width / 2
    
            windowed = np.clip(img, img_min, img_max)
             # 값들을 0과 1 사이로 재조정
            windowed = (windowed-img_min) / (img_max - img_min + 1e-7)
            result = (windowed*255).astype(np.uint8)
            # print(f"Applied windowing: [{img_min:.1f}, {img_max:.1f}] → [0, 255]")
            return result
        
        else : # z-score normalization(for MR)
            mean = np.mean(img)
            std = np.std(img)
    
            normalized = (img - mean) / (std + 1e-7) 
            result = (normalized*255).astype(np.uint8)
            # print(f"Applied normalization: [{mean:.1f}, {std:.1f}] → [0, 255]")
            return result 
            
            
    '''
    Resize 3D volumn to target size
    '''
    def resize_volume_3d(self, volume):
        current_shape = volume.shape
        target_shape = (self.target_depth,self.target_height, self.target_width)
        if current_shape == target_shape:
    
            return volume
    
        else:
            # 3D resizing using scipy.ndimage
            # 각 축별로 얼마나 확대/축소할 지 비율을 계산
            zoom_factors = [target_shape[i] / current_shape[i] for i in range(3)]
    
            # resize with linear interpolation
            resized_volume = ndimage.zoom(volume, zoom_factors, order = 1, mode='nearest')
    
            # 리사이즈 후 shape이 목표보다 클 수 있으므로 슬라이싱으로 정확한 목표 shape에 맞춘다.
            resized_volume = resized_volume[:self.target_depth, :self.target_height, :self.target_width]
    
            # 필요한 경우 패딩 추가 
    
            # uint8 타입(0~255 범위)으로 변환해서 반환 -> 
            return resized_volume.astype(np.uint8)

    '''
    Process DICOM series and return as Numpy array (for Kaggle : no file saving)
    '''
    def process_series(self,series_path):
        try:
            datasets, series_name = self.load_dicom_series(series_path)

            # Check first DICOm to determine 2D/3D 
            # 3D 볼륨인지, 2D 슬라이스인지 확인하기 
            first_ds = datasets[0]
            first_img = first_ds.pixel_array

            # DICOM 파일이 1개이고 그 안에 여러 프레임(3D)이 있으면 3D DICOM으로 처리한다. 
            if len(datasets)==1 and first_img.ndim == 3:
                # 1. Single 3D DICOM file
                return self._process_single_3d_dicom(first_ds, series_name)
            else:
                # 2. Multiple 2D DICOM files
                # 여러 개의 2D 파일 슬라이스가 있으면 각각을 읽어서 3D 볼륨을 쌓는 방식으로 처리한다.
                return self._process_multiple_2d_dicoms(datasets, series_name)
        # 파일을 읽거나 처리 중 오류가 발생하면 예외를 발생시킨다.
        except Exception as e:
            print(f"Failed to process series {series_path}: {e}")
            raise 

    '''
    Process single 3D DICOM file(for kaggle : no file saving)
    한 파일에 여러 슬라이스가 들어있는 CT/MRI를 전처리하여 3D numpy 배열로 반환하는 함수 
    '''
    def _process_single_3d_dicom(self, ds, series_name):
        # get pixel array
        volume = ds.pixel_array.astype(np.float32)
        # print(f'volume.shape : {volume.shape}')

        # Apply RescaleSlope and RescaleIntercept

        slope = getattr(ds, 'RescaleSlope',1)
        intercept = getattr(ds,'RescaleIntercept',0)

        if slope != 1 or intercept != 0:
            volume = volume * float(slope) + float(intercept)
            #print(f"Applied rescaling: slope={slope}, intercept={intercept}")

        
        # Get windowing settings
        window_center,  window_width = self.get_windowing_params(ds)

        # Apply windowing to each slice
        processed_slices = []

        print(f'volume.shape : {volume.shape}')

        for i in range(volume.shape[0]): # 여기서 volumn.shape의 결과와 순서는 ?
            slice_img = volume[i]
            processed_img = self.apply_windowing_or_normalize(slice_img, window_center, window_width)
            processed_slices.append(processed_img)
            del slice_img, processed_img
        gc.collect()

        volume = np.stack(processed_slices, axis = 0)
        print(f"3D volume shape after windowing: {volume.shape}")
        del processed_slices
        gc.collect()

        # 3D resize
        final_volume = self.resize_volume_3d(volume)
        del volume
        gc.collect()
        
        print(f"Successfully processed 3D DICOM series {series_name}")
        return final_volume
    
    '''
    Process multiple 2D dicom files
    '''

    def _process_multiple_2d_dicoms(self, datasets,series_name):
        slice_info = self.extract_slice_info(datasets) # extract_slice_info : slice의 i, instance_number, z_position 등을 담은 slice_info 리스트를 반환한다. 
        sorted_slices = self.sort_slices_by_position(slice_info) # sort_slices_by_position : slice_info 리스트를 받아서 'z_position'을 기준으로 정렬 후 정렬된 리스트 반환 
        first_img = self.extract_pixel_array(sorted_slices[0]['dataset']) # 첫번 째 슬라이스를 통해서 
        window_center, window_width = self.get_windowing_params(sorted_slices[0]['dataset'], first_img)
        processed_slices = []

        for slice_data in sorted_slices:
            ds = slice_data['dataset']
            img = self.extract_pixel_array(ds)
            processed_img = self.apply_windowing_or_normalize(img,window_center, window_width)
            resized_img = cv2.resize(processed_img, (self.target_width, self.target_height))
            processed_slices.append(resized_img)
            
            del ds, img, processed_img, resized_img
            gc.collect()
            
        volume = np.stack(processed_slices, axis = 0) # axis = 0의 의미는? 
        
        del processed_slices
        gc.collect()
        
        final_volume = self.resize_volume_3d(volume)
        del volume
        gc.collect()
        
        return final_volume
    
        
    """
    DICOM processing function for Kaggle inference (single series)
    
    Args:
        series_path: Path to DICOM series
        target_shape: Target volume size (depth, height, width)
    
    Returns:
        np.ndarray: Processed volume
    """
def process_dicom_series_kaggle(series_path, target_shape = (32,384,384)):
    preprocessor = DICOMPreprocessor(target_shape = target_shape)
    return preprocessor.process_series(series_path)
        

    '''
    Safe DICOM processing with memory cleanup
    '''
def process_dicom_series_safe(series_path, target_shape = (32,384,384)):
    try:
        volume = process_dicom_series_kaggle(series_path, target_shape)
        return volume
    finally:
        # memory cleanup
        gc.collect()

    '''
    Test function 
    - Text processing for single series
    '''
def test_single_series(series_path, target_shape = (32, 384, 384)):
    try:
        print(f'process_Dicom_seires호출 : {series_path}')
        volume = process_dicom_series_safe(series_path, target_shape)
        print(f'test_single_series : {volume.shape}')
        return volume
    except Exception as e:
        print(f"✗ Failed to process series: {e}")
        return None

In [23]:
path = os.path.join(SERIES_PATH,'/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647' )
volume = test_single_series(path)

In [22]:
print(volume)

(32, 384, 384)


## 전처리하는데 메모리가 부족하다
- del, gc 활용

## del 
- 변수나 객체에 대한 참조를 삭제한다.
- 객체의 참조만 끊을 뿐 실제 메모리 해제는 gc가 결정한다.
### gc
- 자동으로 해제되지 않는 객체를 수동으로 정리할 수 있게 해준다.
- gc.collect()를 호출하면 즉시 가비지 컬렉션을 실행하여 더이상 참조되지 않는 객체를 메모리에서 해제한다.

### 전처리하는데 시간이 너무 오래 걸린다.  -> 병렬 처리 
- 병렬 처리
- 병렬 처리해도 append가 잘 될까? 

DICOM 파일을 더 빠르게 로딩하려면 **병렬 처리(멀티스레딩/멀티프로세싱)**를 활용하는 것이 가장 효과적입니다.  
pydicom의 dcmread는 순차적으로 파일을 읽으면 느릴 수 있으나, 여러 파일을 동시에 읽으면 I/O 대기 시간을 줄일 수 있습니다.

***

### 병렬 처리 예시 (concurrent.futures 사용)

아래는 Python의 `concurrent.futures.ThreadPoolExecutor`를 사용해 DICOM 파일을 병렬로 읽는 코드 예시입니다.

```python
import concurrent.futures
import pydicom

def safe_dcmread(filepath):
    try:
        return pydicom.dcmread(filepath)
    except Exception as e:
        print(f"Failed to load {filepath}: {e}")
        return None

def load_dicom_series_parallel(dicom_files):
    datasets = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        results = list(executor.map(safe_dcmread, dicom_files))
    # None이 아닌 것만 필터링
    datasets = [ds for ds in results if ds is not None]
    return datasets

# 기존 코드에서
datasets = load_dicom_series_parallel(dicom_files)
if not datasets:
    raise ValueError(f"No valid DICOM files in {series_path}")
return datasets, series_name
```

- `max_workers=8`은 CPU 코어 수에 맞게 조정 가능 (8~16 추천)
- 실패한 파일은 None으로 반환, 이후 필터링

***

### 추가 팁

- **멀티프로세싱**(ProcessPoolExecutor)도 가능하지만, pydicom 객체는 프로세스 간 공유가 어려울 수 있으니 ThreadPoolExecutor가 더 간단합니다.
- SSD 환경에서 병렬 I/O 효과가 더 큽니다.
- tqdm 등으로 진행률 표시도 추가 가능

***

### 요약

- DICOM 파일 로딩을 병렬화하면 2~4배 이상 속도 향상을 기대할 수 있습니다.
- 위 코드를 기존 for문 대신 사용하면, 대량의 DICOM 파일을 훨씬 빠르게 읽을 수 있습니다.[1][4]

[1](https://www.reddit.com/r/learnpython/comments/1ez4b7d/what_is_a_better_approach_to_opening_multiple/)
[2](https://tistory.hu-nie.com/entry/DICOM%EC%9D%98-%EB%AA%A8%EB%93%A0-%EA%B2%83-2)
[3](https://bo-10000.tistory.com/60)
[4](https://www.reddit.com/r/Python/comments/x87oyq/any_trick_to_load_few_big_files_in_python/)
[5](https://coding-nurse.tistory.com/466)
[6](https://blog.naver.com/elmidion/221715462333)
[7](https://89douner.tistory.com/284)

In [ ]:
'''
Preprocessing one sereis 
'''
from concurrent.futures import ProcessPoolExecutor, as_completed


def preprocess_one_series(sereis_path):
    try:
        volume = process_series(series_path)
        return volume
    except Exception as e:
        print(f"failed to preprocess one sereis : {series_path}")
        return None

all_series_id = os.path.basename(series_path)

print(len(all_series_path))
results = []
with ProcessPoolExecutor(max_workers= 8) as executor:
    future_to_path = {executor.submit(preprocess_one_series,path): path for path in all_series_path }
    for future in as_completed(future_to_path):
        result = future_result()
        if result is not None:
            results.append(result)

1001306


In [2]:
# 아주 간단한 3D CNN 모델 정의하기

import torch
import torch.nn as nn
import torch.nn.functional as F

class Simple3DCNN(nn.Module):
     def __init__(self, in_channels=1, num_classes=1):
            super().__init__()
            self.conv1 = nn.Conv3d(in_channels, 8, kernel_size=3, padding=1)
            self.bn1 = nn.BatchNorm3d(8)
            self.pool1 = nn.MaxPool3d(2)
    
            self.conv2 = nn.Conv3d(8, 16, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm3d(16)
            self.pool2 = nn.MaxPool3d(2)
    
            self.fc1 = nn.Linear(16*8*96*96, 64)
            self.fc2 = nn.Linear(64, num_classes)
    def forward(self, x):
        # x: (B, C, D, H, W)
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
      
            
class DicomVolumeDataset(torch.utils.data.Dataset):
    def __init__(self, path_list, labels, preprocessor):
        self.path_list = path_list
        self.labels = labels
        self.preprocessor = preprocessor

    def __len__(self):
        return len(self.path_list)

    def __getitem__(self, idx):
        volume = self.preprocessor.process_series(self.path_list[idx])  # (D, H, W)
        # (D, H, W) → (1, D, H, W)로 reshape해서 모델에 넘김
        volume = torch.tensor(volume, dtype=torch.float32).unsqueeze(0) / 255.0
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return volume, label


# 예시 DICOM 경로와 라벨 리스트
train_paths = list(train_df['SeriesInstanceUID'])   # ["series_folder1", "series_folder2", ...]
train_labels = list(train_df['Aneurysm Present'])   # [0, 1, ...] (동맥류 유무)

preprocessor = DICOMPreprocessor(target_shape=(32,384,384))
train_data = DicomVolumeDataset(train_paths, train_labels, preprocessor)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Simple3DCNN(in_channels=1, num_classes=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(3):  # 몇 번 돌려보기
    model.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(X).squeeze(-1)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
    print(f'epoch {epoch}, loss: {loss.item():.4f}')

def predict(series_path):
    model.eval()
    volume = preprocessor.process_series(series_path)   # (D, H, W)
    X = torch.tensor(volume, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0  # (1, 1, D, H, W)
    X = X.to(device)
    with torch.no_grad():
        logits = model(X)
        prob = torch.sigmoid(logits).item()
    return prob

'''
TEST
'''
# 예시 사용:
test_series_path = train_paths[0:4]
print('test predict:', predict(test_series_path))

class DummyInferenceServer:
    def __init__(self, predict_fn):
        self.predict = predict_fn

    def run_local_gateway(self, series_paths):
        results = []
        for path in series_paths:
            result = self.predict(path)
            results.append({'SeriesInstanceUID': os.path.basename(path), 'Aneurysm Present': result})
        df = pd.DataFrame(results)
        print(df)
        return df

# 서버 객체 생성 및 로컬 테스트
inference_server = DummyInferenceServer(predict)

df = inference_server.run_local_gateway(test_series_paths)



NameError: name 'results' is not defined

### 모델을 훈련시켜보자 

In [ ]:
# train.csv에서 성별 데이터 1,0으로 변경하기
train_copy = train_df.copy()

train_copy['PatientSex'] = train_copy['PatientSex'].map( {'Female' : 0, 'Male' : 1 })
train_copy

In [ ]:
import gc

BATCH_SIZE = 4 
batch_vols = []
batch_series_uids = []
batch_sexes = []
batch_ages = []
batch_labels= []

model = Simple3DAneurysmCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()
model.train()

for i, series_uid in enumerate(os.listdir(SERIES_PATH)):
    series_path = os.path.join(SERIES_PATH,series_uid)
    vol = volumn_from_series(series_path)
    if vol is None:
        continue

    # train.csv에서 sex,age 정보 가져오기 
    row = train_copy[train_copy['SeriesInstanceUID'] == series_uid]
    if row.empty:
        continue
    sex = float(row['PatientSex'].iloc[0]) # 값 추출 
    age = float(row['PatientAge'].iloc[0]) 
    label = float(row['Aneurysm Present'].iloc[0])
    
    batch_vols.append(vol)
    batch_sexes.append(sex)
    batch_ages.append(age)
    batch_labels.append(label)
    batch_series_uids.append(series_uid)

    # 배치가 찼을 경우 처리
    if len(batch_vols) == BATCH_SIZE:
        
        batch_array = np.stack(batch_vols, axis = 0)
        '''
        여기서 batch_vols들이 모두 동일한 shpae이어야 stack할 수 있음
        '''
        # 모델 입력 
        clinical_features = list(zip(batch_sexes, batch_ages))

        # print(f"torch.from_numpy(batch_array) : {torch.from_numpy(batch_array).shape}")
        # torch.Size([4, 32, 384, 384])
        tensor_image = torch.from_numpy(batch_array).unsqueeze(1).float() 
        #print(f"tensor_image.shape : {tensor_image.shape}")
        # torch.Size([4, 1, 32, 384, 384])

        
        # print(f"torch.tensor(clinical_features) : {torch.tensor(clinical_features)}")
    
        tensor_clinical = torch.tensor(clinical_features).float()  # batch 크기 맞춰서 (1,2)

        tensor_labels = torch.tensor(batch_labels).float()
        tensor_labels = tensor_labels.unsqueeze(1)

        print(f"tensor_labels : {tensor_labels.shape}" )
        
        # 모델 호출
        optimizer.zero_grad()
        outputs = model(tensor_image, tensor_clinical)
        loss = criterion(outputs, tensor_labels)
        loss.backward()
        optimizer.step()

        # 메모리 정리
        del batch_vols , batch_sexes, batch_ages, batch_series_uids
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        batch_vols = []
        batch_sexes = []
        batch_ages = []
        batch_labels = []
        batch_series_uids = []

# ❌ 메모리 부족 문제 발생함 !! 
- Your notebook tried to allocate more memory than is available. It has restarted.

- 배치 단위로 처리할 수 있다. 

### batch_array = np.stack(batch_vols, axis = 0) 에서 발생
- all input arrays must have the same shape

- volumn shape가 vol.shape : (1, 25, 528, 528) 인 것이 있음

- 왜 4차원인게 있는거지??

### 발생 에러
- vol = np.stack(slices, axis=0) -> ValueError: need at least one array to stack

### 이제 Dicom 이미지를 series별 volumn을 생성함 
- 이제 train.csv를 통해 동맥류 여부를 예측하는 모델을 만들어보자

In [ ]:
'''
매우 간단한 3D CNN 모델 
'''
import torch, torch.nn as nn
class Simple3DAneurysmCNN(nn.Module):
    def __init__(self, num_classes=1, num_clinical_features = 2):
        super().__init__()

        # 이미지 경로 
        self.conv1 = nn.Conv3d(1,8,kernel_size = 3, padding= 1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool3d((1,1,1))
        self.fc_img = nn.Linear(8, 16)

        # 임상 정보 경로
        self.fc_clinical = nn.Linear(num_clinical_features, 8)

        # 이미지 + 임상 정보 합친 후 최종 분류
        self.fc_combined = nn.Linear(16 + 8, num_classes)
    
    def forward(self,x_img, x_clinical):
        # 이미지 처리 
        x= self.relu(self.conv1(x_img))
        x= self.pool(x)
        x = x.view(x.size(0),-1)
        img_feat = self.fc_img(x)

        # 임상 정보 처리 
        clin_feat = self.relu(self.fc_clinical(x_clinical))

        # 두 경로 합치기 
        combined = torch.cat([img_feat, clin_feat], dim=1)
        out = torch.sigmoid(self.fc_combined(combined))
        return out

# Dataset 클래스 생성 

In [ ]:
from torch.utils.data import Dataset

class RSNA_Dataset(Dataset):
    def __init__(self,df):
        self.df = df
        self.series_root_path = SERIES_PATH
        self.series_uids = df['SeriesInstanceUID']

    def __len__(self):
        return len(self.series_uids)
        
    def __getitem__(self, idx):
        series_uid = self.series_uids[idx]
        series_path = f"{self.series_root_path}/{series_uid}"

        volume = volumn_from_series(series_path)

        row = self.df[self.df['SeriesInstanceUID']==series_uid]
        sex = float(row['PatientSex'])
        age = float(row['PatientAge'])
        labels = float(row['Aneurysm Present'])


        tensor_image = torch.from_numpy(volume).unsqueeze(0).float()
        tensor_clinical = torch.tensor([sex, age]).float()
        tensor_label = torch.tensor(labels).float()

        return tensor_image, tensor_clinical , tensor_label
         

In [ ]:
from torch.utils.data import DataLoader

train_dataset = RSNA_Dataset(train_copy)
val_dataset = RSNA_Dataset(val_df)

train_loader = DataLoader(train_dataset, batch_size = 4, shuffle= True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size = 4, shuffle= True,pin_memory=True)

In [ ]:
model = Simple3DAneurysmCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.BCELoss()
num_epochs = 5

for epoch in range(num_epochs):
    
    model.train()
    for images, clinicals, labels in train_loader:
        labels = labels.unsqueeze(1)
        print(train_loader)
        optimizer.zero_grad()
        outputs = model(images, clinicals)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
   
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total = 0
        for images, clinicals, labels in val_loader:
            outputs = model(images, clinicals)
            preds = (outputs > 0.5).float()
            total_correct += (preds == labels).sum().item()
            total += preds.numel()

        accuracy = total_correct / total
        print(f"Epoch {epoch+1} Validation Accuracy: {accuracy:.4f}")

# 모델의 정확도를 확인해보자 
- 모델의 예측값과 정답을 비교하기 

In [ ]:
model.eval() # 모델 평가 모드 

with torch.no_grad():
    total_correct = 0
    total = 0
    for images, clinicals, labels in val_loader:
        outputs = model(images,clinicals)
        preds = (outputs >0.5).float()
        total_correct += (preds == labels).sum().item()
        total += preds.numel()
    accuracy = total_correct / total
print(f"Epoch {epoch+1} Validation Accuracy: {accuracy:.4f}")
        

In [ ]:
def get_3d_array(series_uid):
    # 시리즈 폴더 경로
    series_path_uid = os.path.join(SERIES_PATH,series_uid)
    
    # DICOM 파일 이름 추출 후 정렬 (InstanceNumber 또는 FileName 기반 정렬)

    # os.listdir(series_path_uid) # 해당 디렉토리 내의 모든 파일과 디렉토리 리스트를 리턴한다. 
    
    dcm_files = [f for f in os.listdir(series_path_uid)]
    
    # # 인스턴스 번호 기준 정렬 (선택적)
    # dcm_files.sort(key=lambda x: pydicom.dcmread(os.path.join(series_path, x)).InstanceNumber)
    
    # 모든 DICOM 읽기
    slices = [pydicom.dcmread(os.path.join(series_path_uid, f)) for f in dcm_files]

    print(slices[0].ImagePositionPatient)
    print('Voxelsize', slices[0].Get)
    
    # 3D 배열 생성 (shape: z, y, x)
    volume = np.stack([s.pixel_array for s in slices], axis=0)
    
    # print(volume.shape)  # (슬라이스 수, y크기, x크기)
    return volume

### XYZ <-> IR2 함수

In [ ]:
IrcTuple = collections.namedtuple('IrcTuple',['index','row','col'] )
XyzTuple = collections.namedtuple('XyzTuple',['x','y','z'])

'''
coord_irc : xyz로 변환하려는 irc 좌표
coord_xyz : irc로 변환하려는 xyz 좌표
origin_xyz:  dicom_image.GetOrigin() 
vxSize_xyz :  dicom_image.GetSpacing()
direction_a: dicom_image.GetDicrection()
'''
def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1]
    # IRC 좌표 (I, R, C)를 (C, R, I) 순서로 바꾸어 cri_a에 저장
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coords_xyz = (direction_a @  (cri_a *vxSize_a )) + origin_a
    return XyzTuple(*coords_xyz)

def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ( (coord_a-origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0])) # z,y,x로 반환

## 이미지 클래스 정의하기

- glob.glob(pattern) : 주어진 경로 패턴에 맞는 모든 파일 경로를 리스트로 반환한다.
- data-unversioned/part2/luna/subset*/{}.mhd : data-unversioned/part2/luna/ 폴더 내에서 subset으로 시작하는 모든 서브 폴더가 대상 
- {}.mhd'.format(series_uid) : 그 폴더 내에서 series_uid.mhd 파일을 찾음
- [0] : 찾은 파일 중 첫 번째 결과만 취함


## Image를 담을 클래스 정의

In [ ]:
class ImageClass:
    def __init__(self, series_uid, sod_uid):
        dcm_dir_path = glob.glob(
            SERIES_PATH +'/{}'.format(series_uid)       
        )[0]

        dcm_path = glob.glob(
            dcm_dir_path+"/{}.dcm".format(sod_uid)
        )[0]
        
        dcm_image = sitk.ReadImage(dcm_path)
        dcm_a = np.array(sitk.GetArrayFromImage(dcm_image), dtype= np.float32)

        # hu 
        dcm_a.clip(-1000, 1000, dcm_a)

        self.series_uid= series_uid
        self.sod_uid = sod_uid
        self.hu_a = dcm_a
        self.origin_xyz = XyzTuple(*dcm_image.GetOrigin())
        self.vxSize_xyz = XyzTuple(*dcm_image.GetSpacing())
        self.direction_a = np.array(dcm_image.GetDirection()).reshape(3,3)
    '''
3D 영상 데이터에서 주어진 중심 좌표(center_xyz)를 배열 인덱스 좌표(IRC)로 변환하고,
그 주변의 일정 크기(width_irc)로 잘라낸 작은 3D 조각(chuck)을 반환하는 함수
- 여기서 우리 타깃 동맥류가 잘 잘릴까? 
'''
    def getRawCandidate(self,center_xyz, width_irc):
        center_irc = xyz2irc(center_xyz, self.origin_xyz,
                            self.vxSize_xyz, self.direction_a)
        slice_list = []
        for axis, center_val in enumerate(center_irc):
            start_ndx = int(round(center_val - width_irc[axis]/2))
            end_ndx = int(start_ndx + width_irc[axis])
            assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, center_irc, axis])
            if start_ndx < 0:
                    # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                    #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                    start_ndx = 0
                    end_ndx = int(width_irc[axis])
    
            if end_ndx > self.hu_a.shape[axis]:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                end_ndx = self.hu_a.shape[axis]
                start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])
    
                slice_list.append(slice(start_ndx, end_ndx))
    
            ct_chunk = self.hu_a[tuple(slice_list)]
    
            return ct_chunk, center_irc

def getImage(series_uid, sod_uid):
    return ImageClass(series_uid,sod_uid)
    
    
def getImageRawCandidate(series_uid, sod_uid, center_xyz, width_irc):
    image = getImage(series_uid,sod_uid)
    image_chunk, center_irc = image.getRawCandidate(center_xyz,width_irc)
    return image_chunk,center_irc

In [ ]:
# 테스트 
img = Image('1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647',
           '1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450')

print(img.vxSize_xyz)
print(img.origin_xyz)
print(img.direction_a)

In [ ]:
get_3d_array('1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647')

## CandidateInfoTuple
CandidateInfoTuple = collections.namedtuple('CandidateInfoTuple',
                                series_uid, sop_uid, center_xyz')

In [ ]:
import ast 
### 2D dicom image를 3D array로 불러오기 

'''
여기 코드 에러 있음 ** ,  series_uid만 다르고 다른 항목 데이터 모두 동일함 
'''

# 전체 데이터셋에서 후보 동맥류 정보 리스트를 만드는 함수 
def getCandidateInfoList():

    # 동맥류가 있는 위치 : 정답 동맥류 
    localizers_dict = {}
    candidateInfo_list = []
    with open('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv','r') as f:
        for row in list(csv.reader(f))[1:]: # 인덱스 0에는 열 이름이 있다.그래서 값은 1행부터 있음
            series_uid = row[0] 
            sop_instance_uid = row[1]
            z_value = get_z_position(row)
            data = ast.literal_eval(row[2]) # str을 딕셔너리로 변환하기 
            xyz = tuple([data['x'], data['y'],z_value])
            localizers_dict.setdefault(series_uid, []).append(
                CandidateInfoTuple(True, series_uid, sop_instance_uid, xyz )
            )

    # train_data 보기 
    with open('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv','r') as f:
        for row in list(csv.reader(f))[1:]:
            is_aneurysm_present = bool(int(row[17]))
            
            for localizer in localizers_dict.get(series_uid, []):
                if row[0] == localizer.series_uid:
                    xyz = localizer.center_xyz
                    series_uid = localizer.series_uid
                    sop_uid = localizer.sop_uid
                    candidateInfo_list.append(CandidateInfoTuple(
                        is_aneurysm_present,
                        series_uid,
                        sop_uid,
                        xyz
                    ))
                else:
                    continue

    return candidateInfo_list

def get_z_position(row):
    
    # series에서 seris_uid에 맞는 series 폴더 찾기
    # series폴더에서 sop_uid 슬라이스 찾기
    # 해당 슬라이스가 몇 번째 인지 알기 
    series_folder = os.path.join(SERIES_PATH, row[0])
    series_image = os.path.join(series_folder, row[1]+'.dcm')
    with open (series_image,'rb') as f: # 'rb'로 열어야함 
        ds = pydicom.dcmread(f)
        # InstanceNumber , DICOM 파일 내에서 해당 슬라이스가 시리즈 내에서 몇 번째 위치하는지를 의미함.
        if 'ImagePositionPatient' in ds:
            z_value = ds.ImagePositionPatient[2]            
        # except AttributeError: # 해당 Dicom 파일에 ImagePositionPatient가 아예없는 경우 발생 
        #     print(series_image+'\n')
        else:   
            # MR인 경우 에러 발생함. 
            error_ds = ds
            z_value = 0.0
        return z_value
        


def display_image(array):
    """
    3D Numpy 배열을 슬라이서 위젯을 사용하여 표시 
    """
    # 슬라이드를 사용하여 슬라이스를 스크롤 
    def view_image(slice_index):
        plt.figure(figsize = (10,10)) # 이미지를 표시할 Figure를 설정
        plt.imshow(array[slice_index], cmap='gray') # 현재 슬라이스 이미지를 회색조로 표시 
        plt.title(f'Slice {slice_index}') # 슬라이스 번호를 제목으로 설정
        plt.show() # 이미지를 출력
    # 슬라이더 생성
    slice_slider = widgets.IntSlider(min=0, max= array.shape[0]-1, step = 1, description='Slice:')
    
    widgets.interact(view_image, slice_index = slice_slider)

In [ ]:
candidateInfo_list = getCandidateInfoList()
positiveInfo_list = [x for x in candidateInfo_list if x[0]]

In [ ]:
candidateInfo_list[0]

In [ ]:
positiveInfo_list[0]

In [ ]:
print(len(positiveInfo_list)) # 1863

## RSNA Dataset Class 정의

In [ ]:
class RSNADataSet(Dataset):
    def __init__(self, vel_sride = 0, isValSet_bool = None, series_uid=None):
        self.candidateInfo_list = copy.copy(getCandidateInfoList())
        if series_uid:
            self.candidateInfo_list = [ x for x in self.candidateInfo_list if x.series_uid == series_uid]
    def __len__(self):
        return len(self.candidateInfo_list)
    def __getitem__(self,ndx):
        candidateInfo_tup = self.candidateInfo_list[ndx]
        width_irc = (32,48,48)

        candidate_a, center_irc = getImageRawCandidate(
            candidateInfo_tup[1],
            candidateInfo_tup[2],
            candidateInfo_tup[3],
            width_irc
        )
        candidate_t = torch.from_numpy(candidate_a)
        candidate_t = candidate_t.to(torch.float32)

        pos_t = torch.tensor([
            not candidateInfo_tup.is_aneurysm_present,
            candidateInfo_tup.is_aneurysm_present
        ], dtype = torch.long,)

        return (
            candidate_t,
            pos_t,
            candidateInfo_tup.series_uid,
            torch.tensor(center_irc),
        )

In [ ]:
RSNADataSet()[0][0]

In [ ]:
train_localizers.info()
# trian 데이터에서 동맥류가 있는 환자는 1863명 있는데 왜 여기는 총 데이터가 2254개이지?
# 여기서 1863은 동맥류가 하나 이상 있는 환자 수이다. 
# train_localizers 에는 동맥류 각각의 위치 좌표를 나타낸다. 

train_localizers.SeriesInstanceUID.value_counts()

In [ ]:
train["Aneurysm Present"].sum()  # 1863

###  값 :
- ['SeriesInstanceUID', 'PatientAge', 'PatientSex', 'Modality', 'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery', 'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery', 'Left Middle Cerebral Artery', 'Right Middle Cerebral Artery', 'Anterior Communicating Artery', 'Left Anterior Cerebral Artery', 'Right Anterior Cerebral Artery', 'Left Posterior Communicating Artery', 'Right Posterior Communicating Artery', 'Basilar Tip', 'Other Posterior Circulation', 'Aneurysm Present']

- ['1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647', '64', 'Female', 'MRA', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

### Simpleitk를 활용한 간단한 이미지 처리 Crop

##  Histogram Equalization 

- 이미지의 대비를 향상시키기 위해 사용 : 이미지의 밝기 분포를 조정하여 어두운부분과 밝은 부분의 대비를 더 명확하게 만든다.
- 각 픽셀의 밝기 값을 조정하여 히스토그램의 분포를 균일하게 만드는 프로세스
- 이렇게 하면 어두운 영역에서 세부 정보를 더 잘 볼 수 있게 되어 이미지의 시각적 품질이 향상된다. 

In [ ]:
# 라이브러리 불러오기 

import cv2
import matplotlib.pyplot as plt

In [ ]:
# 이미지 불러오기 


img_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'

ds = pydicom.dcmread(img_path)
img = ds.pixel_array

# 히스토그램 평탄화

img.dtype # dtype('int16')
if img.dtype != 'uint8':
    img = cv2.convertScaleAbs(img, alpha = (255.0/img.max()))

dst = cv2.equalizeHist(img)


In [ ]:
# 히스토그램 평탄화 전 후 plot 비교
plt.figure(figsize = (20,10))
plt.subplot(1,2,1)
plt.imshow(img, cmap = 'gray')
plt.subplot(1,2,2)
plt.imshow(dst, cmap='gray')

## CLAHE (Contrast Limited Adaptive Histogram Equalization)

- Histogram Equalization은 원본 이미지의 전체에 대한 균일화를 진행하기 떄문에 밝아지지 않아도 될 부분이 밝아지거나 , 원래의 형태나 형테를 알아보기 힘든 경우가 발생할 수 있음.

- 이를 개선할 수 있는 방법이 CLAHE 방법이다.

- 이미지를 작은 블록으로 나누고, 각 블록에 대해 히스토그램 평활화를 개별적으로 적용한다.

In [ ]:
# 라이브러리 불러오기 
import cv2
import matplotlib.pyplot as plt

In [ ]:
# CLAHA 객체 생성 및 적용 

# 이미지 불러오기 

# CLAHE 객체 생성
clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize=(12,12))

img2 = clahe.apply(img)

In [ ]:
# 히스토그램 평탄화 전 후 plot 비교

plt.figure(figsize = (20,10))
plt.subplot(1,3,1).imshow(img, cmap='gray')
plt.subplot(1,3,2).imshow(dst, cmap='gray')
plt.subplot(1,3,3).imshow(img2, cmap='gray')

# Image Normalization


### Min-Max Normalization
- 데이터의 최솟값과 최댓값을 활용하여 데이터를 일반화하므로 데이터의 분포가 완전히 바뀌지는 않지만, 일정한 범위로 조절된다. 

In [ ]:
# 이미지 불러오기 
img_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'

ds = pydicom.dcmread(img_path)
img = ds.pixel_array

In [ ]:
min_value = np.min(img)
max_value = np.max(img)

normalized_image = (img - min_value) / (max_value - min_value)

In [ ]:
# Visualization

plt.figure(figsize = (10,4))
plt.subplot(1,2,1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')

plt.subplot(1,2,2)
plt.imshow(normalized_image, cmap='gray')
plt.title('Normalized Image')

plt.tight_layout()
plt.show()

### Z-score Normalization
- 데이터를 평균과 표준 편차를 활용하여 표준화하는 기법이며 데이터의 평균 중심에서 얼마나 떨어져 있는지를 표준 점수로 표현하게 되며, 정규화를 거친 데이터의 평균은 0이 되고 표준 편차는 1이 된다.

In [ ]:
img_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'

ds = pydicom.dcmread(img_path)
img = ds.pixel_array

In [ ]:
# 평균값과 표준 편차를 계산
mean_value = np.mean(img)
std_dev = np.std(img)

# Z-score normalization
normalized_sitk_image =  img - mean_value
normalized_sitk_image /= std_dev

# 정규화된 이미지를 다시 numpy array로 변환
# normalized_image_np = sitk.GetArrayFromImage(normalized_sitk_image)

In [ ]:
# 시각화

plt.figure(figsize = (10,4))
plt.subplot(1,2,1)
plt.imshow(img,cmap='gray')
plt.title('Original Image')

plt.subplot(1,2,2).imshow(normalized_sitk_image, cmap='gray')
plt.title('Z - Normalized Image')

plt.tight_layout()
plt.show()

# 분석 흐름 정리

In [ ]:
import pandas as pd
import os 

DATA_PATH = '/kaggle/input/rsna-intracranial-aneurysm-detection'
SERIES_PATH = os.path.join(DATA_PATH,'series')
SEG_PATH = os.path.join(DATA_PATH,'segmentations')
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
train_localizers_df = pd.read_csv(os.path.join(DATA_PATH, 'train_localizers.csv'))

- 시리즈 폴더에는 각 환자별 sub폴더가 있고, 각 폴더에 3D 슬라이스 DICOM들이 있다.

### DICOM을 3D numpy array로 stack하여 읽는 방법

In [ ]:
# DICOM 폴더 내의 모든 DICOM 파일을 3D 이미지로 읽기
# load_dicom_series 

#DICOM 이미지를 3D Numpy 배열로 변환
#dicom_to_numpy(dicom_images)

# 3D Numpy 배열을 표시
#display_image(dicom_array)

In [ ]:
# 여기서 모든 series에 대해서 수행해야함

series_folder = os.path.join(SERIES_PATH, str(train_df.loc[0,'SeriesInstanceUID']))

# str(train_df.loc[0,'SeriesInstanceUID'] : train_df의 첫번째 샘플의 id값을 가져온다. 

# SeriesInstanceUID 로 DICOM 표준에서 한 시리즈를 고유하게 식별 가능하다! 

vol = load_dicom_series(series_folder)
dicom_array = dicom_to_numpy(vol)
print(dicom_array.shape)

In [ ]:
df = pd.DataFrame({'A':[1,2,3], 'B':[4,5,6]},index = ['x','y','z'])

df.iloc[0,1] # 위치로 접근 
df.loc['x','B'] # 라벨로 접근

### 데이터 전처리

In [ ]:
# 데이터 전처리
import cv2

# CLAHE 적용
def process_clahe(img):
    clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize=(12,12))
    clahe.apply(img)
    return clahe.apply(img)

In [ ]:
def process_min_max_normalize(img):
    min_value = np.min(img)
    max_value = np.max(img)

    normalized_image = (img - min_value) / (max_value - min_value)
    return normalized_image

def process_z_score_normalize(img):
    mean_value = np.mean(img)
    std_dev = np.std(img)

    # Z-score normalization
    normalized_sitk_image =  img - mean_value
    normalized_sitk_image /= std_dev
    return normalized_sitk_image

In [ ]:
train_df.info()

In [ ]:
import numpy as np
X , y = [],[]
for idx, row in train_df.iterrows():
    series_folder = os.path.join(SERIES_PATH, str(train_df.loc[idx,'SeriesInstanceUID']))
    files = sorted(os.listdir(series_folder))
    dcm = pydicom.dcmread(os.path.join(series_folder, files[0]))
    img = dcm.pixel_array
   # img = process_min_max_normalize(img)
    X.append(img)
    y.append(row['Aneurysm Present'])
X

## Segmentation 데이터 살펴보기
- /kaggle/input/rsna-intracranial-aneurysm-detection/segmentations
- nii 

In [ ]:

segmentation_dir = '/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations'
dir = '/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations/1.2.826.0.1.3680043.8.498.10035643165968342618460849823699311381.nii'
image = sitk.ReadImage(dir)
x,y, z =image.GetSize() # (512, 512, 228)
image.ImagePositionPatient

# 그러면 train_localizers에서 x,y좌표에 + z좌표를 한후 xyz2IRC를 하면되나?

## 모델 한번 돌려보기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


# 간단한 CNN 모델 예시
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 128 * 128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        x = self.features(x)
        print(x.shape)  # feature map shape 확인
        x = x.view(x.size(0), -1)
    
        x = self.classifier(x)
        return x

# 데이터셋
images = RSNADataSet() # 이미지 데이터 (이미 준비된 텐서)
dataloader = DataLoader(images, batch_size=8, shuffle=True)
labels = torch.tensor([1, 0], dtype=torch.float32) # 라벨

# 모델 인스턴스
model = SimpleCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 예시
model.train()
for epoch in range(10):
    for batch in dataloader:
        images, labels, _, _ = batch
        labels = labels[:, 1].float()  # 양성 라벨만 선택

        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

### 